In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
import pandas as pd

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 1. 모델 및 토크나이저 로드
MODEL_NAME = "skt/kogpt2-base-v2"  # 예시: KoGPT-2 모델
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# pad_token이 없는 경우 pad_token을 eos_token으로 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # pad_token을 eos_token으로 설정

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

In [8]:
# 2. 데이터셋 로드 및 전처리
class SentenceRefinementDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=256):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]["input_text"]  # 어색한 문장
        target_text = self.data.iloc[idx]["refined_text"]  # 다듬어진 문장

        # "정제하기:" 프롬프트 추가하여 문장 다듬기 작업을 명확히 함
        input_text = f"정제하기: {input_text}"

        # 토큰화
        encoding = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )

        # KoGPT2는 causal LM이라 labels에 패딩을 제외하기 위해 -100을 사용
        labels = target_encoding["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100  # 패딩 부분을 -100으로 설정

        # input_ids와 labels 크기 맞추기
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

In [9]:
# 3. 데이터 로드
dataset = SentenceRefinementDataset("./dataset/sentence_refinement.csv", tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [12]:
# 4. 모델 학습 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
# 5. 학습 루프
epochs = 15
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # 모델에 입력 및 학습
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader)}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
device = "cuda"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(64128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [27]:
# 테스트할 입력 데이터
input_text = "나는 오늘처럼 마음 약한 사람이 되고 싶어서 기분은 절대 그렇게 해주질 않는"

# 입력 텍스트 토크나이징
input_ids = tokenizer.encode(f"정제하기: {input_text}", return_tensors="pt").to(device)

In [28]:
# 5. 모델을 사용해 예측하기
model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=256, num_beams=4, early_stopping=True)

# 6. 결과 텍스트 디코딩
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 7. 결과 출력
print("입력 텍스트:", input_text)
print("모델 출력 텍스트:", output_text)

입력 텍스트: 나는 오늘처럼 마음 약한 사람이 되고 싶어서 기분은 절대 그렇게 해주질 않는
모델 출력 텍스트: 번거로움 마찬가지다창업팀창업팀창업팀창업팀 학생부종합전형창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀 등의 물적ramprampramprampramp 물적ramp 불가피스테 물적 불가피스테 물적ramp 불가피 물적 불가피 물적 불가피 물적 불가피 물적 불가피 물적 불가피 물적 불가피 짙게rampramprampramp창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀창업팀 라임떨 라임떨 라임 자체의 불가피관련기사emp 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜 라임 덜ramp
